In [1]:
import sys
sys.path.append('../')
from stim_surface_code.stability import StabilityPatch
from stim_surface_code import noise

# Basic usage

In [2]:
s = StabilityPatch(6, 6, 6, 'Z')
s

------------------------------------------------------------------
|    |    |X 36|    |    |    |X 37|    |    |    |X 38|    |    |
----------+----+--------------+----+--------------+----+----------
|    |   0|    |   1|    |   2|    |   3|    |   4|    |   5|    |
-----+----+----+----+----+----+----+----+----+----+----+----+-----
|X 39|    |Z 40|    |X 41|    |Z 42|    |X 43|    |Z 44|    |X 45|
-----+----+----+----+----+----+----+----+----+----+----+----+-----
|    |   6|    |   7|    |   8|    |   9|    |  10|    |  11|    |
----------+----+----+----+----+----+----+----+----+----+----------
|    |    |X 46|    |Z 47|    |X 48|    |Z 49|    |X 50|    |    |
----------+----+----+----+----+----+----+----+----+----+----------
|    |  12|    |  13|    |  14|    |  15|    |  16|    |  17|    |
-----+----+----+----+----+----+----+----+----+----+----+----+-----
|X 51|    |Z 52|    |X 53|    |Z 54|    |X 55|    |Z 56|    |X 57|
-----+----+----+----+----+----+----+----+----+----+----+----+-

In [3]:
# 10x better noise vals than Google's device
noise_params = noise.GoogleNoiseParams.improve_noise_params(10)

print(noise_params.baseline_error_means)

{'T1': 0.0002, 'T2': 0.00030000000000000003, 'gate1_err': -5.0, 'gate2_err': -3.5, 'readout_err': -2.7}


In [4]:
s.set_error_vals_normal(noise_params.baseline_error_means, noise_params.baseline_error_stdevs)

In [5]:
s.count_logical_errors()

(1.89e-05, 10000000)

# Increasing d

In [6]:
ds = [4,6,8]
for d in ds:
    s = StabilityPatch(d, d, d)
    s.set_error_vals_normal(noise_params.baseline_error_means, noise_params.baseline_error_stdevs)
    ler = s.count_logical_errors(10**7)[0]
    print(ler)

0.0003805
1.71e-05
8e-07
